In [1]:
import sys
from pathlib import Path
import pandas as pd

# Notebook cell
%load_ext autoreload
%autoreload 2

# Get root directory (assuming notebook is in root/notebooks/)
NOTEBOOK_DIR = Path.cwd()
ROOT_DIR = NOTEBOOK_DIR.parent if NOTEBOOK_DIR.name == 'notebooks' else NOTEBOOK_DIR

# Add src directory to Python path
sys.path.append(str(ROOT_DIR / 'src'))

# Verify path
print(f"Python will look in these locations:\n{sys.path}")


# --- Execute the processor ---
import utils
from config import date_str, DOWNLOAD_DIR, DEST_DIR

path_data = f'..\data\{date_str}_df_finviz_merged.parquet'
# path_corr = '..\data\df_corr_emv_matrix.parquet'
# path_cov = '..\data\df_cov_emv_matrix.parquet'
# path_output = f'..\picks\{date_str}_portf.txt'


print(utils.__file__)  # Should point to your src/utils.py
print(f'path_date: {path_data}')
# print(f'path_corr: {path_corr}')
# print(f'path_cov: {path_cov}')  
# print(f'path_output: {path_output}')
# print((f'date_str: {date_str}'))



Python will look in these locations:
['C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9\\python311.zip', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9\\DLLs', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9\\Lib', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9', 'C:\\Users\\ping\\Files_win10\\python\\py311\\.venv', '', 'C:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages', 'C:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages\\win32', 'C:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages\\win32\\lib', 'C:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages\\Pythonwin', 'c:\\Users\\ping\\Files_win10\\python\\py311\\stocks\\src']


c:\Users\ping\Files_win10\python\py311\stocks\src\utils.py
path_date: ..\data\2025-04-08_df_finviz_merged.parquet


In [2]:
import pandas as pd
# Set pandas display options to show more columns and rows
pd.set_option('display.max_columns', None)  # Show all columns
# pd.set_option('display.max_rows', 10)       # Limit to 10 rows for readability
# pd.set_option('display.width', None)        # Let the display adjust to the window
# pd.set_option('display.max_colwidth', None) # Show full content of each cell
pd.set_option('display.max_rows', 100)
# pd.set_option('display.width', 120)
pd.set_option('display.float_format', '{:.4f}'.format)

In [3]:
import pandas as pd

def get_column_values_above_threshold(df, column_name='Avg Volume, M', threshold=0.75):
  """
  Analyzes the number and percentage of values in a DataFrame column that are above a specified threshold,
  and returns the filtered DataFrame.

  Args:
    df (pd.DataFrame): The input DataFrame.
    column_name (str): The name of the column to analyze. Defaults to 'Avg Volume, M'.
    threshold (float): The threshold value to compare against. Defaults to 1.00.

  Returns:
    pd.DataFrame: A DataFrame containing only the rows where the specified column's value is above the threshold.
  """
  
  count_before = len(df)
  above_threshold_df = df[df[column_name] > threshold]
  count_after = len(above_threshold_df)
  percentage = (count_after / len(df)) * 100

  print(f"count_before: {count_before}")
  print(f"count_after above threshold ({threshold}): {count_after}")
  print(f"Percentage above threshold ({threshold}): {percentage:.2f}%")

  return above_threshold_df


In [4]:
df_data = pd.read_parquet(path_data)

# liquidity filter, Avg Volume, M > 0.75M
df_data = get_column_values_above_threshold(df_data, column_name='Avg Volume, M', threshold=0.75)

# Drop specified columns with NaNs in df_data
df_data = df_data.drop(['All-Time High %', 'All-Time Low %', 'Dividend %'], axis=1)

# df_corr = pd.read_parquet(path_corr)
# df_cov = pd.read_parquet(path_cov)

# print(f'\ndf_cov.shape: {df_cov.shape}')
# display(df_cov.head())

# print(f'\ndf_corr.shape: {df_corr.shape}')
# display(df_corr.head())

print(f'\ndf_data.shape: {df_data.shape}')
display(df_data.head())
display((df_data.describe()))

count_before: 1520
count_after above threshold (0.75): 1162
Percentage above threshold (0.75): 76.45%

df_data.shape: (1162, 52)


,Company,Info,"MktCap AUM, M",Beta,RSI,Perf YTD %,Perf 3D %,Perf Week %,Perf Month %,Perf Quart %,Perf Half %,Perf Year %,SMA20 %,SMA50 %,SMA200 %,50D High %,50D Low %,52W High %,52W Low %,ATR,ATR/Price %,Volatility W %,Volatility M %,"Volume, M","Avg Volume, M",Rel Volume,Price,Change %,Sharpe 3d,Sortino 3d,Omega 3d,Sharpe 5d,Sortino 5d,Omega 5d,Sharpe 10d,Sortino 10d,Omega 10d,Sharpe 15d,Sortino 15d,Omega 15d,Sharpe 30d,Sortino 30d,Omega 30d,Sharpe 60d,Sortino 60d,Omega 60d,Sharpe 120d,Sortino 120d,Omega 120d,Sharpe 250d,Sortino 250d,Omega 250d
Ticker,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
MSFT,Microsoft Corporation,"Technology, Software - Infrastructure",2642330.0000,0.9600,27.9300,-15.6700,-4.9717,-7.0000,-6.5000,-16.9200,-14.5700,-16.4700,-6.7400,-10.3500,-15.3600,-20.7300,3.0900,-24.1100,3.0900,11.5700,3.2551,3.7700,2.4300,33.1648,24.2700,1.3700,355.4400,-0.6800,-45.3986,-15.4104,0.0000,-21.4168,-13.3590,0.0000,-9.9048,-9.2276,0.1694,-6.6465,-6.8077,0.3089,-4.0139,-4.7498,0.5155,-2.5504,-3.1528,0.6349,-1.4476,-1.7814,0.7701,-0.8479,-1.0659,0.8646
AAPL,Apple Inc,"Technology, Consumer Electronics",2596840.0000,1.3200,20.1600,-30.9700,-15.1435,-22.5500,-24.0100,-29.4400,-23.7800,1.9400,-18.4100,-23.8600,-24.4000,-30.8500,-1.0000,-33.5400,5.3600,9.4600,5.4723,5.0100,3.3600,116.3159,56.6900,2.0500,172.8700,-4.7300,-74.5277,-15.6975,0.0000,-40.5882,-15.0354,0.0000,-10.8116,-9.4224,0.1331,-6.7924,-6.6444,0.2751,-6.2419,-6.1714,0.3139,-3.2718,-3.5932,0.5485,-2.1082,-2.3875,0.6727,0.1145,0.1503,1.0211
NVDA,NVIDIA Corp,"Technology, Semiconductors",2359720.0000,2.0500,32.5400,-27.9800,-5.4028,-12.2000,-9.6000,-35.2800,-22.5800,9.8900,-13.5800,-19.3900,-23.6700,-32.5800,11.6500,-36.8400,27.9100,7.0500,7.2898,7.7000,4.9100,463.4073,291.2800,1.5900,96.7100,-0.9500,4.8688,17.1963,2.5320,-9.6236,-9.5703,0.2120,-7.5765,-7.5861,0.2501,-6.2167,-6.4373,0.3276,-3.8264,-4.3511,0.5344,-1.7323,-2.0777,0.7429,-0.9794,-1.2219,0.8458,0.3993,0.5493,1.0685
AMZN,Amazon.com Inc,"Consumer Cyclical, Internet Retail",1813170.0000,1.2600,29.9400,-22.0200,-4.3439,-10.9700,-12.0500,-24.8300,-8.2700,-7.5500,-10.9900,-18.4600,-14.5000,-29.4500,6.0200,-29.4500,12.8500,9.6000,5.6111,6.8700,3.7400,85.2975,44.3400,1.9200,171.0900,-2.3800,-0.3624,-0.7022,0.9374,-11.1959,-10.3119,0.1566,-7.5932,-7.4917,0.2590,-4.4343,-4.8343,0.4566,-4.4770,-4.8658,0.4811,-2.9142,-3.3342,0.6138,-0.5681,-0.7337,0.9101,-0.2618,-0.3441,0.9570
GOOGL,Alphabet Inc,"Communication Services, Internet Content & Inf...",1777390.0000,0.9800,28.4900,-23.5600,-3.9942,-7.8800,-12.7600,-26.5000,-13.3800,-5.1100,-9.2000,-16.5700,-16.8700,-30.1100,2.9700,-30.1100,2.9700,5.8300,4.0290,3.9200,3.0500,50.4367,32.9300,1.5300,144.7000,-1.4000,-3.2793,-5.0757,0.5478,-14.7627,-11.7328,0.0873,-10.1136,-9.1235,0.1620,-6.0612,-6.3220,0.3686,-4.3768,-4.8355,0.5109,-3.3902,-3.8320,0.5746,-0.8184,-1.0797,0.8735,-0.2297,-0.3150,0.9611


,"MktCap AUM, M",Beta,RSI,Perf YTD %,Perf 3D %,Perf Week %,Perf Month %,Perf Quart %,Perf Half %,Perf Year %,SMA20 %,SMA50 %,SMA200 %,50D High %,50D Low %,52W High %,52W Low %,ATR,ATR/Price %,Volatility W %,Volatility M %,"Volume, M","Avg Volume, M",Rel Volume,Price,Change %,Sharpe 3d,Sortino 3d,Omega 3d,Sharpe 5d,Sortino 5d,Omega 5d,Sharpe 10d,Sortino 10d,Omega 10d,Sharpe 15d,Sortino 15d,Omega 15d,Sharpe 30d,Sortino 30d,Omega 30d,Sharpe 60d,Sortino 60d,Omega 60d,Sharpe 120d,Sortino 120d,Omega 120d,Sharpe 250d,Sortino 250d,Omega 250d
count,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000
mean,55349.8881,0.9912,29.9453,-11.3058,-8.1158,-11.5498,-10.7893,-12.2877,-11.6618,-1.3276,-10.4863,-13.2043,-11.9940,-22.0466,4.0883,-26.8140,18.6135,4.1109,4.5334,4.9738,3.1552,10.3389,6.4135,1.5431,92.3627,-1.9898,-100.9004,13.9631,2.4306,-17.7992,-10.4015,0.2106,-7.5337,-6.8666,0.3129,-5.5462,-5.4448,0.3993,-3.2355,-3.4707,0.6062,-1.2079,-1.2862,0.8430,-0.9000,-1.0421,0.8669,-0.0291,0.0562,1.0083
std,179310.6326,0.5519,9.0967,15.3473,4.2976,6.3293,8.4613,16.0117,18.2664,41.1873,6.4020,9.4577,13.4789,12.1495,6.0340,14.9096,44.9654,5.3373,2.6126,2.6706,1.7063,27.9172,14.4894,0.7273,105.4818,2.1665,867.4509,126.0040,11.1835,19.3161,24.1741,3.0076,3.7688,4.1041,0.4425,2.9529,2.8952,0.3138,2.1461,2.4479,0.2810,1.7714,2.2600,0.2901,1.1438,1.4893,0.1899,0.8810,1.3041,0.1689
min,2930.0000,-3.3700,12.6200,-75.5200,-32.3031,-48.2800,-54.1400,-76.1200,-77.5100,-85.5700,-50.9400,-62.4200,-74.9000,-76.5300,-8.5500,-95.7400,-8.5500,0.0200,0.0358,0.0100,0.0100,0.5500,0.7528,0.4400,2.0200,-12.6400,-18229.8368,-15.8745,0.0000,-525.3285,-15.8691,0.0000,-20.5219,-12.8259,0.0000,-16.8403,-11.8472,0.0162,-7.7297,-7.8342,0.1607,-6.8200,-6.8037,0.2967,-4.0586,-4.5462,0.4630,-2.7379,-3.1449,0.6045
25%,8225.0000,0.6625,22.9650,-20.9900,-10.1778,-15.0575,-15.7275,-22.2225,-21.9425,-18.7400,-14.0600,-18.9275,-20.2925,-29.3350,0.7900,-36.4750,1.8250,1.0200,3.0037,3.4500,2.2025,1.9714,1.5000,1.0800,30.9150,-3.1475,-40.1204,-15.2873,0.0000,-21.2504,-13.3285,0.0000,-10.0812,-9.0334,0.1130,-7.3615,-7.1468,0.2156,-4.8363,-5.1630,0.4120,-2.5288,-2.9458,0.6294,-1.7473,-2.1296,0.7269,-0.6133,-0.7822,0.8957
50%,16715.0000,0.9800,28.2100,-11.2600,-7.9156,-11.5350,-11.2000,-12.2200,-12.8100,-4.6750,-10.5150,-13.1250,-12.1000,-20.7050,2.7000,-24.9300,6.2650,2.4150,4.0162,4.5200,2.9100,3.9023,2.6500,1.3500,58.9600,-1.6850,-17.9870,-13.4672,0.0000,-15.4085,-11.8652,0.0116,-8.0434,-7.7784,0.1998,-5.8237,-5.9812,0.3154,-3.6449,-4.0905,0.5264,-1.4463,-1.7998,0.7713,-0.9972,-1.2759,0.8335,-0.0815,-0.1114,0.9853
75%,44485.0000,1.2300,35.2275,-1.8475,-5.8853,-7.7400,-5.4325,-1.5750,-1.7850,8.3350,-6.3250,-6.5100,-3.1475,-13.6200,6.0000,-15.9625,21.8250,5.1200,5.6755,6.2100,3.9325,9.1202,5.5200,1.8175,106.8125,-0.5700,-6.1405,-7.9383,0.2928,-10.6358,-9.9062,0.1467,-5.3645,-5.7342,0.3667,-3.7669,-4.2739,0.4956,-1.8110,-2.2841,0.7209,-0.0200,-0.0282,0.9965,-0.1411,-0.1936,0.9753,0.5083,0.7104,1.0928
max,2642330.0000,4.2200,82.4800,70.1000,24.5474,41.4800,38.0500,83.8600,93.2500,822.0000,35.5000,56.8700,45.9800,-0.0100,102.0600,-0.0100,921.6100,46.3100,28.7273,24.7300,16.2500,470.6654,291.2800,7.2900,909.9100,10.7800,62.9372,722.2425,65.3425,17.7720,802.5541,102.1123,10.1807,72.1709,9.9546,6.7615,24.3995,4.0171,6.0289,13.9181,2.9185,4.0598,7.8482,2.1111,2.4871,5.7127,1.8643,3.6528,7.4545,1.8151


In [5]:
import pandas as pd
import os
import datetime

# --- Configuration ---
STATS_HISTORY_FILE = 'market_daily_summary_stats_history.csv' # Where to store the history

def append_daily_stats(df_daily_data, date_str, filename=STATS_HISTORY_FILE):
    """
    Calculates descriptive stats for df_daily_data, flattens them,
    and appends/updates them as a row in a historical CSV file.

    Args:
        df_daily_data (pd.DataFrame): DataFrame containing the raw data for ALL stocks for ONE specific day.
        date_str (str): The date for this data in 'YYYY-MM-DD' format.
        filename (str): The path to the CSV file storing the historical summary stats.
    """
    print(f"Processing stats for date: {date_str}")

    # 1. Calculate descriptive statistics for the input day's data
    try:
        daily_stats_raw = df_daily_data.describe()
        print("Successfully calculated describe() stats.")
    except Exception as e:
        print(f"Error calculating describe() stats: {e}")
        return # Stop if calculation fails

    # 2. Flatten the describe() output into a single row DataFrame
    try:
        # Unstack turns the stats (mean, std, etc.) into columns alongside the original metrics
        stats_flat = daily_stats_raw.unstack().to_frame().T
        # Create clearer column names like 'Metric_Statistic' (e.g., 'RSI_mean', 'SMA50 %_50%')
        stats_flat.columns = ['_'.join(col).strip() for col in stats_flat.columns.values]
        # Add the date to this row
        stats_flat['Date'] = pd.to_datetime(date_str)
        # Set the Date as the index for this single row
        stats_flat = stats_flat.set_index('Date')
        print("Successfully flattened stats into a single row.")
        # print(stats_flat) # Optional: print the flattened stats for verification

    except Exception as e:
        print(f"Error flattening the stats DataFrame: {e}")
        return # Stop if flattening fails

    # 3. Load existing history, append/update, and save
    try:
        df_summary_history = pd.DataFrame() # Initialize empty
        if os.path.exists(filename):
            print(f"Loading existing history from: {filename}")
            df_summary_history = pd.read_csv(filename, index_col='Date', parse_dates=True)

        # Check if the date already exists - update if it does, append if it doesn't
        current_date = pd.to_datetime(date_str)
        if current_date in df_summary_history.index:
            print(f"Updating existing entry for {date_str}")
            # Update existing row(s) - combines columns, replacing old values for this date
            df_summary_history.update(stats_flat)
            # Alternative: df_summary_history.loc[current_date] = stats_flat.iloc[0] # Direct overwrite
        else:
            print(f"Appending new entry for {date_str}")
            df_summary_history = pd.concat([df_summary_history, stats_flat])

        # Ensure the history is sorted by date
        df_summary_history = df_summary_history.sort_index()

        # Save the updated history back to CSV
        df_summary_history.to_csv(filename, index=True) # index=True saves the Date index
        print(f"Successfully saved updated stats history to: {filename}")

    except Exception as e:
        print(f"Error loading/saving the stats history file '{filename}': {e}")



In [6]:
# Simulate processing for a couple of days
print("-" * 20, "Day 1", "-" * 20)
append_daily_stats(df_data, date_str)

-------------------- Day 1 --------------------
Processing stats for date: 2025-04-08
Successfully calculated describe() stats.
Successfully flattened stats into a single row.
Loading existing history from: market_daily_summary_stats_history.csv


Updating existing entry for 2025-04-08


Successfully saved updated stats history to: market_daily_summary_stats_history.csv


In [7]:
print(f'date_str: {date_str}')
df_data.describe()

date_str: 2025-04-08


,"MktCap AUM, M",Beta,RSI,Perf YTD %,Perf 3D %,Perf Week %,Perf Month %,Perf Quart %,Perf Half %,Perf Year %,SMA20 %,SMA50 %,SMA200 %,50D High %,50D Low %,52W High %,52W Low %,ATR,ATR/Price %,Volatility W %,Volatility M %,"Volume, M","Avg Volume, M",Rel Volume,Price,Change %,Sharpe 3d,Sortino 3d,Omega 3d,Sharpe 5d,Sortino 5d,Omega 5d,Sharpe 10d,Sortino 10d,Omega 10d,Sharpe 15d,Sortino 15d,Omega 15d,Sharpe 30d,Sortino 30d,Omega 30d,Sharpe 60d,Sortino 60d,Omega 60d,Sharpe 120d,Sortino 120d,Omega 120d,Sharpe 250d,Sortino 250d,Omega 250d
count,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000,1162.0000
mean,55349.8881,0.9912,29.9453,-11.3058,-8.1158,-11.5498,-10.7893,-12.2877,-11.6618,-1.3276,-10.4863,-13.2043,-11.9940,-22.0466,4.0883,-26.8140,18.6135,4.1109,4.5334,4.9738,3.1552,10.3389,6.4135,1.5431,92.3627,-1.9898,-100.9004,13.9631,2.4306,-17.7992,-10.4015,0.2106,-7.5337,-6.8666,0.3129,-5.5462,-5.4448,0.3993,-3.2355,-3.4707,0.6062,-1.2079,-1.2862,0.8430,-0.9000,-1.0421,0.8669,-0.0291,0.0562,1.0083
std,179310.6326,0.5519,9.0967,15.3473,4.2976,6.3293,8.4613,16.0117,18.2664,41.1873,6.4020,9.4577,13.4789,12.1495,6.0340,14.9096,44.9654,5.3373,2.6126,2.6706,1.7063,27.9172,14.4894,0.7273,105.4818,2.1665,867.4509,126.0040,11.1835,19.3161,24.1741,3.0076,3.7688,4.1041,0.4425,2.9529,2.8952,0.3138,2.1461,2.4479,0.2810,1.7714,2.2600,0.2901,1.1438,1.4893,0.1899,0.8810,1.3041,0.1689
min,2930.0000,-3.3700,12.6200,-75.5200,-32.3031,-48.2800,-54.1400,-76.1200,-77.5100,-85.5700,-50.9400,-62.4200,-74.9000,-76.5300,-8.5500,-95.7400,-8.5500,0.0200,0.0358,0.0100,0.0100,0.5500,0.7528,0.4400,2.0200,-12.6400,-18229.8368,-15.8745,0.0000,-525.3285,-15.8691,0.0000,-20.5219,-12.8259,0.0000,-16.8403,-11.8472,0.0162,-7.7297,-7.8342,0.1607,-6.8200,-6.8037,0.2967,-4.0586,-4.5462,0.4630,-2.7379,-3.1449,0.6045
25%,8225.0000,0.6625,22.9650,-20.9900,-10.1778,-15.0575,-15.7275,-22.2225,-21.9425,-18.7400,-14.0600,-18.9275,-20.2925,-29.3350,0.7900,-36.4750,1.8250,1.0200,3.0037,3.4500,2.2025,1.9714,1.5000,1.0800,30.9150,-3.1475,-40.1204,-15.2873,0.0000,-21.2504,-13.3285,0.0000,-10.0812,-9.0334,0.1130,-7.3615,-7.1468,0.2156,-4.8363,-5.1630,0.4120,-2.5288,-2.9458,0.6294,-1.7473,-2.1296,0.7269,-0.6133,-0.7822,0.8957
50%,16715.0000,0.9800,28.2100,-11.2600,-7.9156,-11.5350,-11.2000,-12.2200,-12.8100,-4.6750,-10.5150,-13.1250,-12.1000,-20.7050,2.7000,-24.9300,6.2650,2.4150,4.0162,4.5200,2.9100,3.9023,2.6500,1.3500,58.9600,-1.6850,-17.9870,-13.4672,0.0000,-15.4085,-11.8652,0.0116,-8.0434,-7.7784,0.1998,-5.8237,-5.9812,0.3154,-3.6449,-4.0905,0.5264,-1.4463,-1.7998,0.7713,-0.9972,-1.2759,0.8335,-0.0815,-0.1114,0.9853
75%,44485.0000,1.2300,35.2275,-1.8475,-5.8853,-7.7400,-5.4325,-1.5750,-1.7850,8.3350,-6.3250,-6.5100,-3.1475,-13.6200,6.0000,-15.9625,21.8250,5.1200,5.6755,6.2100,3.9325,9.1202,5.5200,1.8175,106.8125,-0.5700,-6.1405,-7.9383,0.2928,-10.6358,-9.9062,0.1467,-5.3645,-5.7342,0.3667,-3.7669,-4.2739,0.4956,-1.8110,-2.2841,0.7209,-0.0200,-0.0282,0.9965,-0.1411,-0.1936,0.9753,0.5083,0.7104,1.0928
max,2642330.0000,4.2200,82.4800,70.1000,24.5474,41.4800,38.0500,83.8600,93.2500,822.0000,35.5000,56.8700,45.9800,-0.0100,102.0600,-0.0100,921.6100,46.3100,28.7273,24.7300,16.2500,470.6654,291.2800,7.2900,909.9100,10.7800,62.9372,722.2425,65.3425,17.7720,802.5541,102.1123,10.1807,72.1709,9.9546,6.7615,24.3995,4.0171,6.0289,13.9181,2.9185,4.0598,7.8482,2.1111,2.4871,5.7127,1.8643,3.6528,7.4545,1.8151


In [8]:
print(f'date_str: {date_str}')
df_data.head()

date_str: 2025-04-08


,Company,Info,"MktCap AUM, M",Beta,RSI,Perf YTD %,Perf 3D %,Perf Week %,Perf Month %,Perf Quart %,Perf Half %,Perf Year %,SMA20 %,SMA50 %,SMA200 %,50D High %,50D Low %,52W High %,52W Low %,ATR,ATR/Price %,Volatility W %,Volatility M %,"Volume, M","Avg Volume, M",Rel Volume,Price,Change %,Sharpe 3d,Sortino 3d,Omega 3d,Sharpe 5d,Sortino 5d,Omega 5d,Sharpe 10d,Sortino 10d,Omega 10d,Sharpe 15d,Sortino 15d,Omega 15d,Sharpe 30d,Sortino 30d,Omega 30d,Sharpe 60d,Sortino 60d,Omega 60d,Sharpe 120d,Sortino 120d,Omega 120d,Sharpe 250d,Sortino 250d,Omega 250d
Ticker,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
MSFT,Microsoft Corporation,"Technology, Software - Infrastructure",2642330.0000,0.9600,27.9300,-15.6700,-4.9717,-7.0000,-6.5000,-16.9200,-14.5700,-16.4700,-6.7400,-10.3500,-15.3600,-20.7300,3.0900,-24.1100,3.0900,11.5700,3.2551,3.7700,2.4300,33.1648,24.2700,1.3700,355.4400,-0.6800,-45.3986,-15.4104,0.0000,-21.4168,-13.3590,0.0000,-9.9048,-9.2276,0.1694,-6.6465,-6.8077,0.3089,-4.0139,-4.7498,0.5155,-2.5504,-3.1528,0.6349,-1.4476,-1.7814,0.7701,-0.8479,-1.0659,0.8646
AAPL,Apple Inc,"Technology, Consumer Electronics",2596840.0000,1.3200,20.1600,-30.9700,-15.1435,-22.5500,-24.0100,-29.4400,-23.7800,1.9400,-18.4100,-23.8600,-24.4000,-30.8500,-1.0000,-33.5400,5.3600,9.4600,5.4723,5.0100,3.3600,116.3159,56.6900,2.0500,172.8700,-4.7300,-74.5277,-15.6975,0.0000,-40.5882,-15.0354,0.0000,-10.8116,-9.4224,0.1331,-6.7924,-6.6444,0.2751,-6.2419,-6.1714,0.3139,-3.2718,-3.5932,0.5485,-2.1082,-2.3875,0.6727,0.1145,0.1503,1.0211
NVDA,NVIDIA Corp,"Technology, Semiconductors",2359720.0000,2.0500,32.5400,-27.9800,-5.4028,-12.2000,-9.6000,-35.2800,-22.5800,9.8900,-13.5800,-19.3900,-23.6700,-32.5800,11.6500,-36.8400,27.9100,7.0500,7.2898,7.7000,4.9100,463.4073,291.2800,1.5900,96.7100,-0.9500,4.8688,17.1963,2.5320,-9.6236,-9.5703,0.2120,-7.5765,-7.5861,0.2501,-6.2167,-6.4373,0.3276,-3.8264,-4.3511,0.5344,-1.7323,-2.0777,0.7429,-0.9794,-1.2219,0.8458,0.3993,0.5493,1.0685
AMZN,Amazon.com Inc,"Consumer Cyclical, Internet Retail",1813170.0000,1.2600,29.9400,-22.0200,-4.3439,-10.9700,-12.0500,-24.8300,-8.2700,-7.5500,-10.9900,-18.4600,-14.5000,-29.4500,6.0200,-29.4500,12.8500,9.6000,5.6111,6.8700,3.7400,85.2975,44.3400,1.9200,171.0900,-2.3800,-0.3624,-0.7022,0.9374,-11.1959,-10.3119,0.1566,-7.5932,-7.4917,0.2590,-4.4343,-4.8343,0.4566,-4.4770,-4.8658,0.4811,-2.9142,-3.3342,0.6138,-0.5681,-0.7337,0.9101,-0.2618,-0.3441,0.9570
GOOGL,Alphabet Inc,"Communication Services, Internet Content & Inf...",1777390.0000,0.9800,28.4900,-23.5600,-3.9942,-7.8800,-12.7600,-26.5000,-13.3800,-5.1100,-9.2000,-16.5700,-16.8700,-30.1100,2.9700,-30.1100,2.9700,5.8300,4.0290,3.9200,3.0500,50.4367,32.9300,1.5300,144.7000,-1.4000,-3.2793,-5.0757,0.5478,-14.7627,-11.7328,0.0873,-10.1136,-9.1235,0.1620,-6.0612,-6.3220,0.3686,-4.3768,-4.8355,0.5109,-3.3902,-3.8320,0.5746,-0.8184,-1.0797,0.8735,-0.2297,-0.3150,0.9611
